In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from dateutil import parser

In [22]:
habits_dict = [
    {"habitName": "Leetcode", "habit_id": "1", "habit_created_at": "2022-11-23", 'goal': 5},
    {"habitName": "Gym", "habit_id": "2", "habit_created_at": "2022-12-28", 'goal': 5},
    {"habitName": "GRE", "habit_id": "3", "habit_created_at": "2022-11-25", 'goal': 5},
]

completed_json = [
    {"habit_id": "1", "date": "2022-10-25", "completed": True},
    {"habit_id": "1", "date": "2022-10-26", "completed": True},
    {"habit_id": "1", "date": "2022-10-27", "completed": True},
    {"habit_id": "2", "date": "2022-10-27", "completed": True},
    {"habit_id": "1", "date": "2022-11-30", "completed": True},
    {"habit_id": "1", "date": "2022-12-08", "completed": True},
    {"habit_id": "1", "date": "2022-12-19", "completed": True},
    {"habit_id": "1", "date": "2022-12-20", "completed": True},
    {"habit_id": "2", "date": "2022-12-20", "completed": True},
    {"habit_id": "2", "date": "2022-12-19", "completed": True},
    {"habit_id": "2", "date": "2022-12-20", "completed": True},
    {"habit_id": "1", "date": "2022-12-21", "completed": True},
    {"habit_id": "1", "date": "2022-12-22", "completed": True},
]


In [29]:
def format_week(date):
    if pd.isnull(date):
        return None
    start_of_week = date - pd.to_timedelta(date.weekday(), unit='d')
    end_of_week = start_of_week + pd.to_timedelta(6, unit='d')
    week_string = start_of_week.strftime('%a, %b %d') + ' - ' + end_of_week.strftime('%a, %b %d')
    return week_string

# Load the data into a dataframe, setting the date column as the index
df = pd.DataFrame(completed_json)
df_habits = pd.DataFrame(habits_dict)

# Convert the date column to a datetime index
df['date'] = pd.to_datetime(df['date'])

# Group the data by habit and week, and count the number of days each habit was completed
df_weekly = df.groupby(['habit_id', pd.Grouper(key='date', freq='W')])['date'].count().reset_index(name='completed')

# # Merge in the goal for each habit
df_weekly = df_weekly.merge(df[['habit_id']].drop_duplicates(), on='habit_id')

# Rename the columns and set the index to the week ending
df_weekly = df_weekly.rename(columns={'date': 'week_ending'})

# Sort the data by week ending
df_weekly = df_weekly.sort_index()

df_weekly = df_weekly.merge(df_habits, on='habit_id', how='right')
df_weekly

,habit_id,week_ending,completed,habitName,habit_created_at,goal
0,1,2022-10-30,3.0,Leetcode,2022-11-23,5
1,1,2022-12-04,1.0,Leetcode,2022-11-23,5
2,1,2022-12-11,1.0,Leetcode,2022-11-23,5
3,1,2022-12-25,4.0,Leetcode,2022-11-23,5
4,2,2022-10-30,1.0,Gym,2022-12-28,5
5,2,2022-12-25,3.0,Gym,2022-12-28,5
6,3,NaT,NaN,GRE,2022-11-25,5


In [30]:

# df_weekly['week_ending'] = df_weekly['week_ending'].astype(str)
# result = df_weekly.groupby('week_ending_str').apply(lambda x: x.to_dict(orient='records')).to_dict()
# import json
# with open('results.json', 'w') as f:
#     json.dump(result, f)


In [31]:
now = pd.Timestamp.now()
one_month_ago = now - pd.Timedelta(weeks=6)
date_range = pd.date_range(start=one_month_ago, end=now)

# Group the dates by week and get the last day of each week
df_weeks = date_range.to_frame().resample('W').last()
df_weeks = df_weeks.reset_index()
df_weeks = df_weeks.rename(columns={'index': 'week_ending'})
df_weeks = df_weeks.drop(columns=[df_weeks.columns[1]])

df_weeks

,week_ending
0,2022-11-27
1,2022-12-04
2,2022-12-11
3,2022-12-18
4,2022-12-25
5,2023-01-01
6,2023-01-08


In [33]:
df_merged = df_weekly.merge(df_weeks, on='week_ending', how='right')

# Get the unique week ending values
week_endings = df_merged['week_ending'].unique()

# Loop over the week ending values
for week_ending in week_endings:
    # Filter df_merged to only include rows for the current week ending
    df_filtered = df_merged[df_merged['week_ending'] == week_ending]
    
    # Extract the unique habit_ids from the filtered DataFrame
    habit_ids = df_filtered['habit_id'].unique()

    # Find the difference between the set of all habit_ids and the set of habit_ids in the filtered DataFrame
    missing_habit_ids = set(df_habits['habit_id']) - set(habit_ids)

    # Use the missing_habit_ids to filter the df_habits DataFrame and extract the rows corresponding to the missing habits
    missing_habits = df_habits[df_habits['habit_id'].isin(missing_habit_ids)]
    
    # Set the 'completed' column to 0, the 'goal' column to 0, and the 'total' column to 7 for the missing habits
    missing_habits.loc[:, ['completed', 'week_ending']] = [0, week_ending]
    
    # Append the missing habits to the df_merged DataFrame
    df_merged = pd.concat([df_merged, missing_habits])

df_merged['week_ending_str'] = df_merged['week_ending'].apply(format_week)
df_merged.dropna(subset=['habit_id'], inplace=True)
df_merged = df_merged.sort_values(by=['week_ending', 'habit_id'])

# Calculate the total number of days possible for each week
df_merged['total'] = df_merged['goal'].apply(lambda x: 7 if x > 0 else 0)

df_merged

/tmp/ipykernel_17533/1875718379.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_habits.loc[:, ['completed', 'week_ending']] = [0, week_ending]
/tmp/ipykernel_17533/1875718379.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_habits.loc[:, ['completed', 'week_ending']] = [0, week_ending]
/tmp/ipykernel_17533/1875718379.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

,habit_id,week_ending,completed,habitName,habit_created_at,goal,week_ending_str,total
0,1,2022-11-27,0.0,Leetcode,2022-11-23,5.0,"Mon, Nov 21 - Sun, Nov 27",7
1,2,2022-11-27,0.0,Gym,2022-12-28,5.0,"Mon, Nov 21 - Sun, Nov 27",7
2,3,2022-11-27,0.0,GRE,2022-11-25,5.0,"Mon, Nov 21 - Sun, Nov 27",7
1,1,2022-12-04,1.0,Leetcode,2022-11-23,5.0,"Mon, Nov 28 - Sun, Dec 04",7
1,2,2022-12-04,0.0,Gym,2022-12-28,5.0,"Mon, Nov 28 - Sun, Dec 04",7
2,3,2022-12-04,0.0,GRE,2022-11-25,5.0,"Mon, Nov 28 - Sun, Dec 04",7
2,1,2022-12-11,1.0,Leetcode,2022-11-23,5.0,"Mon, Dec 05 - Sun, Dec 11",7
1,2,2022-12-11,0.0,Gym,2022-12-28,5.0,"Mon, Dec 05 - Sun, Dec 11",7
2,3,2022-12-11,0.0,GRE,2022-11-25,5.0,"Mon, Dec 05 - Sun, Dec 11",7
0,1,2022-12-18,0.0,Leetcode,2022-11-23,5.0,"Mon, Dec 12 - Sun, Dec 18",7
